In [ ]:
import pandas as pd
import requests
import boto3
import json

In [ ]:
#hide
from nbdev.showdoc import *

# retrieval from last FM

> Pull all tracks from last FM

In [ ]:
# export
def last_cred():

    secret_name = "last_keys"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    last_key = json.loads(get_secret_value_response['SecretString'])['last_apiKEY']
    last_agent = json.loads(get_secret_value_response['SecretString'])['last_userAGENT']
    last_user = json.loads(get_secret_value_response['SecretString'])['last_username']

    return last_key, last_agent, last_user

In [ ]:
API_KEY, USER_AGENT, USERNAME = last_cred()

In [ ]:
def last_get(method, period='1month', agent=USER_AGENT, user=USERNAME, key=API_KEY):

    headers = {
    'user-agent': agent
    }

    payload = {'method': method, 
                'period': period,
            'user': user,
            'api_key': key,
            'format': 'json'
                }


    r = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
    print(r.status_code)
    return r.json()

In [ ]:
tracks = last_get('user.gettoptracks')
df_tracks = pd.DataFrame(tracks['toptracks']['track'])
df_tracks = df_tracks[['name', 'artist', 'playcount']]
df_tracks['artist'] = df_tracks['artist'].apply(lambda x: x['name'])
df_tracks['playcount'] = df_tracks['playcount'].astype(int)
df_tracks = df_tracks.sort_values('playcount', ascending=False)


200


In [ ]:
cutoff = 7
df_tracks = df_tracks[df_tracks['playcount'] >= cutoff]

In [ ]:
df_tracks.to_csv('s3://spotify-net/df_tracks.csv')